In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import tensorflow as tf
import cifar10
from cifar10 import num_classes
import matplotlib.pyplot as plt
cifar10.maybe_download_and_extract()

/home/uttaran/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/uttaran/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Data has apparently already been downloaded and unpacked.


In [3]:
train_data,train_labels,onehot_train=cifar10.load_training_data()
test_data,test_labels,onehot_test=cifar10.load_test_data()
train_data=train_data.astype('float32')
train_labels=train_labels.astype('int32')
test_data=test_data.astype('float32')
test_labels=test_labels.astype('int32')
tf.logging.set_verbosity(tf.logging.ERROR)

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_1'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_2'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_3'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_4'>
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/data_batch_5'>
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch
<_io.BufferedReader name='data/CIFAR-10/cifar-10-batches-py/test_batch'>


In [4]:
weight_decay = 1e-2
regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
filter_size = 3
feature_maps = 96
acc_track = []
l=1

In [5]:
#Convert to GrayScale

train_data_gray = 0.2989 * train_data[:,:,:,0] + 0.5870 * train_data[:,:,:,1] + 0.1140 * train_data[:,:,:,2]

test_data_gray = 0.2989 * test_data[:,:,:,0] + 0.5870 * test_data[:,:,:,1] + 0.1140 * test_data[:,:,:,2]


In [6]:
def cnn_model_fn(features, labels, mode):
      input_layer = tf.reshape(features["x"], [-1, 32, 32,l])
      dropout0_1 = tf.contrib.layers.dropout(inputs=input_layer, keep_prob=0.8)  
      conv1 = tf.contrib.layers.conv2d(
          inputs = dropout0_1,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )
      
      conv1_bn =  tf.contrib.layers.batch_norm(
            inputs=conv1,
            fused = True
      )
      dropout1_2 = tf.contrib.layers.dropout(inputs=conv1_bn, keep_prob=0.7)  
      conv2 = tf.contrib.layers.conv2d(
          inputs = dropout1_2,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv2_bn =  tf.contrib.layers.batch_norm(
            inputs=conv2,
            fused = True
        )
      dropout2_3 = tf.contrib.layers.dropout(inputs=conv2_bn, keep_prob=0.7)

      conv3 = tf.contrib.layers.conv2d(
          inputs = dropout2_3,
          num_outputs = feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          stride = 2
          )
      conv3_bn =  tf.contrib.layers.batch_norm(
            inputs=conv3,
            fused = True
        )
      dropout3_4 = tf.contrib.layers.dropout(inputs=conv3_bn, keep_prob=0.7)  
      conv4 = tf.contrib.layers.conv2d(
          inputs = dropout3_4,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )

      conv4_bn =  tf.contrib.layers.batch_norm(
            inputs=conv4,
            fused = True
        )

      
      #pool2 = tf.contrib.layers.max_pool2d(inputs = conv4_bn,kernel_size = [2,2],stride = 2)
    
      
        #dropout2 =  tf.contrib.layers.dropout(inputs=pool2, keep_prob=0.7)

      dropout4_5 = tf.contrib.layers.dropout(inputs=conv4_bn, keep_prob=0.7)
      
      conv5 = tf.contrib.layers.conv2d(
          inputs = dropout4_5,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          )


      conv5_bn =  tf.contrib.layers.batch_norm(
            inputs=conv5,
            fused = True
        )
      dropout5_6 = tf.contrib.layers.dropout(inputs=conv5_bn, keep_prob=0.7)
      
      conv6 = tf.contrib.layers.conv2d(
          inputs = dropout5_6,
          num_outputs = 2*feature_maps,
          kernel_size = filter_size,
          weights_regularizer=regularizer,
          stride = 2
          )

      conv6_bn =  tf.contrib.layers.batch_norm(
            inputs=conv6,
            fused = True
        )

      
      #pool3 = tf.contrib.layers.max_pool2d(inputs = conv6_bn,kernel_size = [2,2] ,stride = 2)
      dropout3 =  tf.contrib.layers.dropout(inputs=conv6_bn, keep_prob=0.5)
      
#       print("**************************************")
#       print(dropout3.shape)
      pool3_flat = tf.reshape(dropout3, [-1, 8* 8* 192])
    
      logits = tf.layers.dense(inputs=pool3_flat, units=10)
      predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.contrib.layers.softmax(logits)
  }
      if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

      # Calculate Loss (for both TRAIN and EVAL modes)
      loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

      # Configure the Training Op (for TRAIN mode)
      if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

      # Add evaluation metrics (for EVAL mode)
      eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels=labels, predictions=predictions["classes"]),
      }
      return tf.estimator.EstimatorSpec(
          mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [7]:
def current_loss(flag):
    train_eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data_gray},
      y=train_labels,
      shuffle=False)
    eval_result1=cifar_classifier.evaluate(input_fn=train_eval_input_fn)
    if flag:
        print("%%%%%%% Train accuracy %%%%%%%%%%%%\n",eval_result1)
    [a,b,c] = eval_result1.items()

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": test_data_gray},
      y=test_labels,
      num_epochs=1,
      shuffle=False)
    eval_result2=cifar_classifier.evaluate(input_fn=eval_input_fn)
    if flag:
        print("######### Test accuracy #############\n",eval_result2)
    [d,e,f] = eval_result2.items()
    acc_track.append([a[1],d[1]])
    return a[1],d[1]

In [8]:
# Evaluate the model and print results
cifar_classifier = tf.estimator.Estimator(
  model_fn=cnn_model_fn, model_dir="./CIFAR_Alldrop_gray")

# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)
print("started\n")
# Train the model
i = 0 
while 1:
    if i:
        curr_result = current_loss(1)
        print(curr_result)
        if curr_result[0] - curr_result[1] > 0.10 or curr_result[1] > 0.82:
            break
# Train the model
    print("Training")
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data_gray},
      y=train_labels,
      batch_size=256,
      num_epochs=None,
      shuffle=True)

    cifar_classifier.train(
      input_fn=train_input_fn,
      steps=1000,
      hooks=None)
    i=1
    #print(curr_result)
    
print ("Finished")

started

Training
%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.7605, 'loss': 0.7233796, 'global_step': 84000}
######### Test accuracy #############
 {'accuracy': 0.6728, 'loss': 1.1382068, 'global_step': 84000}
(0.7605, 0.6728)
Training


KeyboardInterrupt: 

In [10]:
current_loss(1)

%%%%%%% Train accuracy %%%%%%%%%%%%
 {'accuracy': 0.76428, 'loss': 0.7076295, 'global_step': 84001}
######### Test accuracy #############
 {'accuracy': 0.6784, 'loss': 1.0821195, 'global_step': 84001}


(0.76428, 0.6784)